In [1]:
import os, glob
import numpy as np
import pandas as pd
from scipy import stats
from datetime import date, time, datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,  accuracy_score

w_list= glob.glob("../result/merge_weather/*.csv")
p_list = glob.glob("../result/flower_price_byweek/*")
select_t = [(1,0),(3,2),(2,1),(2,3),(2,4)]
st, flower = select_t[0]
fn_weather = w_list[st]
fn_price = p_list[flower]

print(fn_weather, fn_price)



Using TensorFlow backend.


../result/merge_weather\C0F9L0_后里.csv ../result/flower_price_byweek\Anthurium_pbyweek.csv


In [121]:
class fp_pre:
    def __init__(self,fn_weather, fn_price):
        self.r_data = pd.read_csv(fn_weather, encoding="utf-8")
        self.lc_d = pd.read_csv("../lunar_celeb_date.csv", encoding="utf-8")
        self.df_pbw = pd.read_csv(fn_price, encoding="utf-8")
        self.d_data = self.r_data["Date"]
        
    def prep_rdata(self):
        r_data = self.r_data
        col_fix =  ['T.Max', 'T.Min', 'Precp','Temperature', 'RH', 'StnPres', 'WS', 'WSGust']
        for col in col_fix:
            old = r_data[col]
            new = []
            for i in range(len(old)):
                try:
                    new.append(float(old[i]))
                    tmp = float(old[i])
                except ValueError:
                    new.append(tmp)
            r_data[col] = new
        d_tmp = r_data['T.Max'] - r_data['T.Min']
        r_data["d_tmp"]= d_tmp
        drop_c =["Date",'T.Max', 'T.Min']
        r_data = r_data.drop(columns=drop_c)
        
        def to_zscore(df):
            col_x = df.columns.to_list()
            mean = df.mean(axis=0)
            std = df.std(axis=0)
            for i in range(len(col_x)):
                df[col_x[i]]=(df[col_x[i]]-mean[i])/std[i]
            return  df , (mean, std)
        r_data, self.recordz = to_zscore(r_data)
        self.r_data = r_data
        
    def join_weather(self):
        self.r_data["Date"] = self.d_data
        self.r_data = self.r_data.join(self.lc_d.set_index("date"), on="Date")
        self.r_data = self.r_data.drop(columns="Date")
    
    def shift_rdata(self, shift1, shift2):
        dfs =[]
        arr_all =[]
        if shift2==0:
            df_s = self.r_data.copy()
        else:
            df_s = self.r_data.copy()
            df_s = df_s.shift(periods=shift2)
        arr_all = np.array(df_s)

        for i in range(1,shift1):
            tp = np.array(df_s.shift(periods=i))
            arr_all = np.concatenate((arr_all, tp), axis=1)
        df_all = pd.DataFrame(arr_all)
        df_all["date"] = self.d_data
        df_all = df_all.dropna()
        df_all = df_all.reset_index()
        df_all = df_all.drop(columns="index")
        self.d_data = df_all["date"]
        self.r_data = df_all
    
    def cutyw_rdata(self):
        def trans_to_y_w(d_date):
            year = int(d_date.split("-")[0])
            d_day = date(year,int(d_date.split("-")[1]), int(d_date.split("-")[2]))- date(year, 1, 1)
            d_w = 1+ (d_day.days // 7)
            if d_w ==53:
                d_w = 52
            result =  str(year) + "_" + str(d_w)

            if d_day.days % 7 ==6:
                return result
            else:
                return np.nan
        y_w = []
        for i in range(len(self.d_data)):
            y_w.append(trans_to_y_w(self.d_data[i]))
        self.r_data["y_w"] = y_w
        self.r_data = self.r_data.dropna()
        self.r_data = self.r_data.reset_index()
        self.r_data = self.r_data.drop(columns="index")
    
    def join_pbw_r_data(self):
#         df_join = self.df_pbw.join(self.r_data.set_index("y_w"), on="y_w")
        df_join = self.r_data.join(self.df_pbw.set_index("y_w"), on="y_w")
        df_join = df_join.dropna()
        df_join = df_join.reset_index()
        df_join = df_join.drop(columns="index")
        self.r_data = df_join
    
#     def extract_xy(self):
        
    

In [122]:
fp = fp_pre(fn_weather, fn_price)

In [123]:
fp = fp_pre(fn_weather, fn_price)
fp.prep_rdata()
fp.join_weather()
fp.shift_rdata(7,1)
fp.cutyw_rdata()
fp.join_pbw_r_data()
print(fp.r_data.shape)
fp.r_data.head(10)

(7967, 162)


,0,1,2,3,4,5,6,7,8,9,...,152,153,date,y_w,year,week,w_avg,w_sale,market,price_diff
0,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,14.103072,25593,台中,0.085380
1,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,14.103072,25593,台中,0.085380
2,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,18.129620,50375,台北,-0.016389
3,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,14.103072,25593,台中,0.085380
4,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,14.103072,25593,台中,0.085380
5,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,18.129620,50375,台北,-0.016389
6,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,15.006880,24577,台南,0.315341
7,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,14.103072,25593,台中,0.085380
8,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,14.103072,25593,台中,0.085380
9,-1.641689,-0.299353,0.750366,1.141477,2.539443,1.674389,-1.235394,1.0,0.0,1.0,...,0.0,0.0,2014-01-14,2014_2,2014,2,18.129620,50375,台北,-0.016389
